In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import expit

# download the data from minst
the following cell will download the data auto..

In [1]:
!wget http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
# teh following links are of the test data set
!wget http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
!wget http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz


--2020-10-12 04:46:20--  http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 172.67.171.76, 104.28.6.204, 104.28.7.204, ...
Connecting to yann.lecun.com (yann.lecun.com)|172.67.171.76|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9912422 (9.5M) [application/x-gzip]
Saving to: ‘train-images-idx3-ubyte.gz’

train-images-idx3-u 100%[===================>]   9.45M  1.08MB/s    in 10s     

2020-10-12 04:46:31 (933 KB/s) - ‘train-images-idx3-ubyte.gz’ saved [9912422/9912422]

--2020-10-12 04:46:31--  http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
Resolving yann.lecun.com (yann.lecun.com)... 172.67.171.76, 104.28.6.204, 104.28.7.204, ...
Connecting to yann.lecun.com (yann.lecun.com)|172.67.171.76|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28881 (28K) [application/x-gzip]
Saving to: ‘train-labels-idx1-ubyte.gz’

train-labels-idx1-u 100%[===================>]  28.20K   131K

In [2]:
ls -l

total 22688
-rw-r--r--  1 vaibhavpatel  staff     1044 Oct 12 04:45 Digit_classification.ipynb
-rw-r--r--  1 vaibhavpatel  staff       35 Oct 12 04:16 README.md
-rw-r--r--  1 vaibhavpatel  staff  1648877 Jul 22  2000 t10k-images-idx3-ubyte.gz
-rw-r--r--  1 vaibhavpatel  staff     4542 Jul 22  2000 t10k-labels-idx1-ubyte.gz
-rw-r--r--  1 vaibhavpatel  staff  9912422 Jul 22  2000 train-images-idx3-ubyte.gz
-rw-r--r--  1 vaibhavpatel  staff    28881 Jul 22  2000 train-labels-idx1-ubyte.gz


In [3]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np
    
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [7]:
X_train, y_train = load_mnist("./",kind="train")
X_test, y_test = load_mnist("./",kind="t10k")
print("X-Train set size",np.shape(X_train))
print("y-Test set size", np.shape(X_test))

print("X - Label set Size", np.shape(y_train))
print("y - Label set size", np.shape(y_test))

X-Train set size (60000, 784)
y-Test set size (10000, 784)
X - Label set Size (60000,)
y - Label set size (10000,)
